FINAL PROJECT NOTEBOOK - GROUP

## Brandon Hoover
#### Data Set - CrimeData & 2020 Census Population

Links to Datasets: 

Arrest Data: https://data.wprdc.org/dataset/arrest-data/resource/e03a89dd-134a-4ee8-a2bd-62c40aeebc6f

Neighborhood Population: https://data.wprdc.org/dataset/2020-census-redistricting-data-extracts/resource/a8414ed5-c50f-417e-bb67-82b734660da6


#### SubMetric - CrimeRate

Theory: It is known that a neighborhood with a low crime rate is more favorable than a neighborhood with a high crime rate. Under that assumption when evalutating the best neighborhood we chose to incorporate the rate of crimes per capita for every neighborhood in Pittsburgh. To accomplish this goal we are using two datasets, Pittsburgh Arrest Data and 2020 Census Data to gather two pieces of information which are the number of arrests and total population of each neighborhood. 

The reason for requiring both pieces of information is that while Southside might initally appear to have a higher Crimerate than Bloomfield, its relative to the amount of people living there since an increase in population enevitably leads to more arrests. So to balance the data we are going to be dividing the *number of arrests* by the *total population* of each respective neighborhood with the result being our submetric we are going to aptly named **CrimeRate**.


Implementation:


